In [ ]:
# %% [markdown]
# # Test de génération d'art
# 
# Ce notebook teste la génération d'œuvres d'art avec les modèles entraînés.

# %%
import sys
sys.path.append('..')

from src.generation import ArtGenerator
from src.validation import ArtValidator
from config.settings import config
from pathlib import Path
import json



In [ ]:
# %%
# Charger le générateur SVG
svg_model_dir = Path("models/svg_model_final")
if svg_model_dir.exists():
    svg_generator = ArtGenerator(str(svg_model_dir), config)
    print("Générateur SVG chargé")
else:
    print(f"Modèle SVG non trouvé: {svg_model_dir}")



In [ ]:
# %%
# Générer des œuvres SVG
if 'svg_generator' in locals():
    print("Génération d'œuvres SVG...")
    svg_artworks = svg_generator.generate_svg(
        prompt="<svg width='800' height='600'>",
        num_return_sequences=3
    )
    
    for i, svg in enumerate(svg_artworks):
        filename = f"generated_svg_{i+1}.svg"
        with open(filename, 'w') as f:
            f.write(svg)
        print(f"Œuvre {i+1} sauvegardée: {filename} (taille: {len(svg)} caractères)")



In [ ]:
# %%
# Charger le générateur G-code
gcode_model_dir = Path("models/gcode_model_final")
if gcode_model_dir.exists():
    gcode_generator = ArtGenerator(str(gcode_model_dir), config)
    print("Générateur G-code chargé")
else:
    print(f"Modèle G-code non trouvé: {gcode_model_dir}")



In [ ]:
# %%
# Générer des œuvres G-code
if 'gcode_generator' in locals():
    print("\nGénération d'œuvres G-code...")
    gcode_artworks = gcode_generator.generate_gcode(
        prompt="G21\nG90",
        num_return_sequences=2
    )
    
    for i, gcode in enumerate(gcode_artworks):
        filename = f"generated_gcode_{i+1}.gcode"
        with open(filename, 'w') as f:
            f.write(gcode)
        print(f"Œuvre {i+1} sauvegardée: {filename} ({len(gcode.splitlines())} lignes)")



In [ ]:
# %%
# Valider les œuvres générées
validator = ArtValidator(config)

print("\nValidation des œuvres SVG générées:")
if 'svg_artworks' in locals():
    for i, svg in enumerate(svg_artworks):
        result = validator.validate(svg, "svg")
        status = "✓" if result.is_valid else "✗"
        print(f"Œuvre SVG {i+1}: {status} "
              f"(erreurs: {len(result.errors)}, avertissements: {len(result.warnings)})")

print("\nValidation des œuvres G-code générées:")
if 'gcode_artworks' in locals():
    for i, gcode in enumerate(gcode_artworks):
        result = validator.validate(gcode, "gcode")
        status = "✓" if result.is_valid else "✗"
        print(f"Œuvre G-code {i+1}: {status} "
              f"(erreurs: {len(result.errors)}, avertissements: {len(result.warnings)})")



In [ ]:
# %%
# Générer avec contraintes
print("\nGénération avec contraintes...")
if 'svg_generator' in locals():
    constrained_svg = svg_generator.generate_conditional(
        format_type="svg",
        style_prompt="geometric minimalist art",
        constraints={
            "shapes": ["circle", "rect", "line"],
            "colors": ["black", "white", "red"],
            "complexity": "low"
        }
    )
    
    with open("constrained_art.svg", 'w') as f:
        f.write(constrained_svg)
    
    print("Œuvre contrainte générée: constrained_art.svg")



In [ ]:
# %%
# Batch generation
print("\nGénération par lot...")
if 'svg_generator' in locals():
    batch = svg_generator.batch_generate(
        num_pieces=5,
        format_type="svg",
        output_dir="data/generated"
    )
    
    print(f"Généré {len(batch)} œuvres dans data/generated/")



In [ ]:
# %%
# Analyser les métriques de validation
if 'svg_artworks' in locals():
    artworks_list = []
    for i, svg in enumerate(svg_artworks):
        artworks_list.append({
            "id": f"svg_{i+1}",
            "format": "svg",
            "content": svg
        })
    
    validation_results = validator.validate_batch(artworks_list)
    
    print("\nRésumé de validation par lot:")
    print(f"  Total: {validation_results['total']}")
    print(f"  Valides: {validation_results['valid']}")
    print(f"  Taux de validité: {validation_results['valid']/validation_results['total']*100:.1f}%")